In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [2]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
import LinearAlgebra: ldiv!, dot
using Test
using ForwardDiff
using SpecialFunctions
using BenchmarkTools
using TransportMap
using QuadGK
using AdaptiveTransportMap: derivative, vander, transform!, evaluate, ncoeff
using DiffResults
using Distributions
using Random
using LoopVectorization
using Optim
using NLsolve
using Polynomials
using Optim: InverseDiagonal#, ldiv!, dot

┌ Info: Precompiling AdaptiveTransportMap [bdf749b0-1400-4207-80d3-e689c0e3f03d]
└ @ Base loading.jl:1260


### Greedy fit

In [3]:
Nx = 10

m = 20
Ne = 100
X = randn(Nx, Ne).^2 + 0.1*randn(Nx, Ne)

10×100 Array{Float64,2}:
  0.0499666    1.55799    0.0492889  …  0.0491313  0.314532    0.731691
  0.00997715   0.347253   2.03541       6.50166    0.0226924  -0.0182651
 -0.0346133    0.5011     1.08153       0.743501   2.50786     0.23986
  0.354064     0.152687   0.11812       0.307737   7.38013    -0.220454
  2.35642     -0.14022    1.25989       0.124705   0.15449     2.04571
  0.0696629    0.432744   0.864013   …  2.97236    1.0575      0.0275118
  4.60394      3.59949    0.697723      2.84641    4.25739     0.013259
  2.69612      0.0907148  0.992818      6.38263    0.0442027   2.42817
  0.963167     2.60901    1.18017       1.15176    3.24779     0.599211
  1.44305      0.0639408  2.2941        0.0277365  0.0657557   1.08061

In [4]:
# @code_warntype Hk_old = HermiteMapk(m, Nx; α = 1e-6)

ErrorException: expression is not a function call, or is too complex for @code_warntype to analyze; break it down to simpler parts if possible

In [5]:
L = LinearTransform(X; diag = true)
transform!(L, X)

10×100 Array{Float64,2}:
 -0.714857   0.558277  -0.715429   …  -0.715563  -0.4915    -0.139317
 -0.868089  -0.584441   0.835291       4.59139   -0.857395  -0.89184
 -0.731851  -0.385398  -0.0100242     -0.228634   0.912399  -0.554346
 -0.460673  -0.606685  -0.631749      -0.494263   4.63372   -0.877239
  1.14216   -0.87521    0.256129      -0.661141  -0.637074   0.891097
 -0.623897  -0.320084   0.0407868  …   1.80497    0.202692  -0.659167
  2.20599    1.57194   -0.259767       1.09657    1.98724   -0.691829
  1.71416   -0.679454   0.149316       5.10098   -0.722185   1.46799
  0.025954   1.19121    0.179594       0.159481   1.64347   -0.231726
  0.141941  -0.594738   0.596546      -0.614077  -0.593768  -0.0516608

In [6]:
X_train = X[:,1:80]
X_valid = X[:,81:100]

10×20 Array{Float64,2}:
  2.3674      0.189346   1.64223    …  -0.715563  -0.4915    -0.139317
 -0.319042    1.24161   -0.609242       4.59139   -0.857395  -0.89184
 -0.597342    0.849379   0.613277      -0.228634   0.912399  -0.554346
 -0.306905    1.07542   -0.537313      -0.494263   4.63372   -0.877239
 -0.257703   -0.835556  -0.0339973     -0.661141  -0.637074   0.891097
 -0.68946    -0.843659  -0.584854   …   1.80497    0.202692  -0.659167
 -0.0409656  -0.647274  -0.790512       1.09657    1.98724   -0.691829
  0.207736   -0.515963   2.13647        5.10098   -0.722185   1.46799
 -0.71748    -0.2432     0.324975       0.159481   1.64347   -0.231726
  1.82571    -0.570654   0.441896      -0.614077  -0.593768  -0.0516608

In [ ]:
@btime greedyfit($m, $Nx, $X_train, $X_valid, $10; verbose = false);

In [14]:
@btime greedyfit($m, $Nx, $X_train, $X_valid, $10; verbose = false);

  21.308 ms (122212 allocations: 34.88 MiB)


### Type stability

In [12]:
B = ProHermite(10; scaled = true)
x = randn(1000);

  30.929 ns (2 allocations: 32 bytes)


0.1425484378703077

In [22]:
function timing()
    @btime vander($B, $0, $x)
    @btime $B($7.0)
    @btime exp(-7.0^2/2.0)
end

timing (generic function with 1 method)

In [23]:
timing()

  34.634 μs (167 allocations: 89.45 KiB)
  31.072 ns (2 allocations: 32 bytes)
  1.181 ns (0 allocations: 0 bytes)


2.289734845645553e-11

In [13]:
@btime vander($B, $0, $x);

  32.879 μs (167 allocations: 89.45 KiB)


1000×11 Array{Float64,2}:
 0.553428  -0.402375   -0.184469    0.405972    …  -0.253378    0.2214
 0.63156   -0.0121385  -0.446416    0.0148647      -0.0190327  -0.312725
 0.468272   0.512313    0.065212   -0.377111       -0.0691254   0.292051
 0.538869   0.429493   -0.138983   -0.414635        0.205378    0.265815
 0.61891   -0.176473   -0.402055    0.210277       -0.247556   -0.188874
 0.477694  -0.504926    0.0396091   0.388098    …   0.0331524   0.303991
 0.628303  -0.0911663  -0.434924    0.110872       -0.139027   -0.279335
 0.157936  -0.371884    0.507506   -0.386292       -0.239625   -0.123452
 0.62566   -0.121826   -0.425635    0.14732        -0.181561   -0.253014
 0.585904  -0.32119    -0.289794    0.35397        -0.319655    0.0637283
 0.540661   0.426394   -0.144521   -0.413954    …   0.211663    0.261168
 0.306568  -0.521291    0.410008    0.0231148       0.292783   -0.214368
 0.308224  -0.52215     0.407526    0.0277468       0.295454   -0.209605
 ⋮                        

In [22]:
ImmutablePolynomial{Float64}

ImmutablePolynomial{Float64,N} where N

In [24]:
typeof(ImmutablePolynomial([1.0]))

ImmutablePolynomial{Float64,1}

In [4]:
@code_warntype B(1.0)

Variables
  P::PhyHermite{10}
  x::Float64

Body::Float64
1 ─ %1  = Base.getproperty(P, :Poly)::PhyPolyHermite{10}
│   %2  = Base.getproperty(%1, :P)::ImmutablePolynomial{Float64,N} where N
│   %3  = (%2)(x)::Float64
│   %4  = Core.apply_type(Base.Val, 2)::Core.Compiler.Const(Val{2}, false)
│   %5  = (%4)()::Core.Compiler.Const(Val{2}(), false)
│   %6  = Base.literal_pow(AdaptiveTransportMap.:^, x, %5)::Float64
│   %7  = -%6::Float64
│   %8  = (%7 / 2)::Float64
│   %9  = AdaptiveTransportMap.exp(%8)::Float64
│   %10 = (%3 * %9)::Float64
└──       return %10


In [36]:
@btime greedyfit($m, $Nx, $X_train, $X_valid, $10; verbose = false);

  23.745 ms (139564 allocations: 38.88 MiB)


In [53]:
@code_warntype greedyfit(m, Nx, X_train, X_valid, 15; verbose = false);

Variables
  #unused#::Core.Compiler.Const(AdaptiveTransportMap.var"#greedyfit##kw"(), false)
  @_2::NamedTuple{(:verbose,),Tuple{Bool}}
  @_3::Core.Compiler.Const(AdaptiveTransportMap.greedyfit, false)
  m::Int64
  k::Int64
  X::Array{Float64,2}
  Xvalid::Array{Float64,2}
  maxterms::Int64
  maxpatience::Int64
  verbose::Bool
  @_11::Int64
  @_12::Bool

Body::Tuple{HermiteMapk{_A,1,_B} where _B where _A,Array{Float64,1},Array{Float64,1}}
1 ─ %1  = Base.haskey(@_2, :maxpatience)::Core.Compiler.Const(false, false)
└──       goto #3 if not %1
2 ─       Core.Compiler.Const(:(Base.getindex(@_2, :maxpatience)), false)
│         Core.Compiler.Const(:(%3 isa AdaptiveTransportMap.Int64), false)
│         Core.Compiler.Const(:(%4), false)
│         Core.Compiler.Const(:(goto %9), false)
│         Core.Compiler.Const(:(%new(Core.TypeError, Symbol("keyword argument"), :maxpatience, AdaptiveTransportMap.Int64, %3)), false)
│         Core.Compiler.Const(:(Core.throw(%7)), false)
│         Core.Compi

In [151]:
@code_warntype greedyfit($m, $Nx, $X_train, $X_valid, $15; verbose = false);

ErrorException: syntax: "$" expression outside quote

In [99]:
Hk_old = HermiteMapk(m, Nx; α = 1e-6);

@time S = Storage(Hk_old.I.f, X);

Hk_new, train_error, valid_error = greedyfit(m, Nx, X_train, X_valid, 10; verbose = true);

  0.000038 seconds (42 allocations: 120.516 KiB)
1 terms - Training error: 1.4443504670380543, Validation error: 1.3147907978711455
2 terms - Training error: 1.013555754394048, Validation error: 1.0148284980423214
3 terms - Training error: 0.9740035731615574, Validation error: 0.9536709077333043
4 terms - Training error: 0.968194963388136, Validation error: 0.9269258934472301
5 terms - Training error: 0.8660654654774056, Validation error: 0.8538651976765612
6 terms - Training error: 0.8468938234481649, Validation error: 0.8374329264723278
7 terms - Training error: 0.8118015219715872, Validation error: 0.8007373200002506
8 terms - Training error: 0.808988833791454, Validation error: 0.8016700608672841
9 terms - Training error: 0.8055509920369043, Validation error: 0.8064967607768131
10 terms - Training error: 0.7857644650326624, Validation error: 0.8090534466320567


In [106]:
Hk_old = HermiteMapk(m, Nx; α = 1e-6);

@time S = Storage(Hk_old.I.f, X);

Hk_new, train_error, valid_error = greedyfit(m, Nx, X_train, X_valid, 10; verbose = true);

  0.000057 seconds (42 allocations: 120.516 KiB)
1 terms - Training error: 1.41428818328749, Validation error: 1.435039932873401
2 terms - Training error: 1.011647231546337, Validation error: 1.0765340323252046
3 terms - Training error: 0.954187562314565, Validation error: 1.009496573370243
4 terms - Training error: 0.9532824496069529, Validation error: 1.007776307368724
5 terms - Training error: 0.841380513349966, Validation error: 0.9041126563257862
6 terms - Training error: 0.8299467025145106, Validation error: 0.9037778557535633
7 terms - Training error: 0.8018842783083129, Validation error: 0.8750517929963674
8 terms - Training error: 0.801758584142784, Validation error: 0.8767880843281514
9 terms - Training error: 0.8015539210804201, Validation error: 0.8774101595696208
10 terms - Training error: 0.7995583928098936, Validation error: 0.8823916723449652


In [83]:
Hk_test = deepcopy(Hk_new)
setcoeff!(Hk_test, zero(getcoeff(Hk_new)));

In [85]:
S_test = Storage(Hk_test.I.f, X_train)
coeff_test = getcoeff(Hk_test)

res = Optim.optimize(Optim.only_fg!(negative_log_likelihood!(S_test, Hk_test, X_train)), coeff_test, Optim.BFGS())
Optim.minimizer(res)

10-element Array{Float64,1}:
 -11.217886066730607
  11.50914577979369
  26.61111083798624
 -24.71937861667548
  34.997147393519235
 -17.03501613771925
  11.077321531464895
   0.034843907277600714
   0.0381866111200113
   0.1782552001411759

In [78]:
res = Optim.optimize(Optim.only_fg!(negative_log_likelihood!(S, Hk_old, ens.S)), coeff, Optim.BFGS())
coeffopt = Optim.minimizer(res)

1-element Array{Float64,1}:
 -1.144117388080984

### Hessian computation 

In [22]:
Nx = 4
m = 10
B = MultiBasis(CstProHermite(m; scaled =true), Nx)

Nψ = 14
# idx = rand(0:5,Nψ, Nx)
idx =Matrix([5  2  5  3;
             0  2  2  5;
             5  4  3  4;
             4  1  3  1;
             3  2  4  5;
             0  3  3  3;
             4  2  0  0;
             2  0  2  4;
             4  4  0  5;
             0  4  0  2;
             4  5  5  5;
             0  1  2  3;
             4  2  4  0;
             4  3  2  3]);


coeff = [  2.421476831418713;
          0.23398851530507098;
         -0.5285368375369742;
          0.8947095708709891;
         -0.688462862924509;
          0.23331743780756214;
          2.0098809355359006;
          1.1243983698817437;
         -0.18373487759949592;
          0.24787268168991677;
          0.9041136183543212;
          1.9776434399606355;
          1.1972246121155632;
          0.3847306368698849]

f = ExpandedFunction(B, idx, coeff)

fp = ParametricFunction(f)
R = IntegratedFunction(fp)
H = HermiteMapk(R; α = 1e-6);
Ne = 500

500

In [23]:
sum(idx;dims = 2)

14×1 Array{Int64,2}:
 15
  9
 16
  9
 14
  9
  6
  8
 13
  6
 19
  6
 10
 12

In [24]:
X = randn(Nx, Ne) .* randn(Nx, Ne) 
S = Storage(H.I.f, X; hess = false);

J = 0.0
dJ = zeros(Nψ)
d2J = zeros(Nψ, Nψ)

@time negative_log_likelihood!(J, dJ, coeff, S, H, X)

precond!

  0.002007 seconds (1.53 k allocations: 2.742 MiB)


precond! (generic function with 2 methods)

In [25]:
@btime res = Optim.optimize(Optim.only_fg!(negative_log_likelihood!($S, $H, $X)), $coeff, Optim.LBFGS())

  104.624 ms (130849 allocations: 233.12 MiB)


 * Status: success

 * Candidate solution
    Final objective value:     1.479295e+00

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 1.24e-05 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.82e-06 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.87e-13 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.26e-13 ≰ 0.0e+00
    |g(x)|                 = 2.84e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    28
    f(x) calls:    85
    ∇f(x) calls:   85


In [26]:
struct Preconditioner
    P::Symmetric{Float64}
    
    F::Cholesky{Float64, Matrix{Float64}}
end

function Preconditioner(P::Matrix{Float64})
    return Preconditioner(Symmetric(P), cholesky(Symmetric(P)))
end

ldiv!(x, P::Preconditioner, b) = copyto!(x, P.F \ b)
dot(A::Array, P::Preconditioner, B::Vector) = dot(A, P.P, B)

ErrorException: error in method definition: function LinearAlgebra.ldiv! must be explicitly imported to be extended

In [27]:

# res = Optim.optimize(Optim.only_fg!(negative_loglikelihood!(S, H, X)), coeff, 
#             Optim.LBFGS(P = Preconditioner(precond)))#, precondprep = (P, x) -> precond!(S, H, X)(P, x)))

In [28]:
@btime res = Optim.optimize(Optim.only_fg!(negative_log_likelihood!($S, $H, $X)), $coeff, Optim.LBFGS())#, precondprep = (P, x) -> Preconditioner(precond!($S, $H, $X)($P.P.data, x))))

  104.817 ms (130849 allocations: 233.12 MiB)


 * Status: success

 * Candidate solution
    Final objective value:     1.479295e+00

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 1.24e-05 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.82e-06 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.87e-13 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.26e-13 ≰ 0.0e+00
    |g(x)|                 = 2.84e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    28
    f(x) calls:    85
    ∇f(x) calls:   85


In [29]:
precond = zeros(Nψ, Nψ)
coeff0 = randn(Nψ)
precond!(precond, coeff, S, H, X)
P = Preconditioner(precond)

res = Optim.optimize(Optim.only_fg!(negative_log_likelihood!(S, H, X)), coeff0, Optim.LBFGS(; m = 20, P = Preconditioner(precond)))

 * Status: success

 * Candidate solution
    Final objective value:     1.479295e+00

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 6.21e-05 ≰ 0.0e+00
    |x - x'|/|x'|          = 9.10e-06 ≰ 0.0e+00
    |f(x) - f(x')|         = 7.68e-13 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 5.19e-13 ≰ 0.0e+00
    |g(x)|                 = 3.43e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    8
    f(x) calls:    17
    ∇f(x) calls:   17


In [30]:
Optim.minimizer(res)

14-element Array{Float64,1}:
  0.2042411208812283
 -0.160665456718217
  5.371746068389059
 -0.19909085135432283
  6.822999267856531
 -0.14374191048398882
 -0.3861327468309733
  0.788767701800107
  2.22362622541736
 -0.9700837073808384
  2.7757966850015574
  0.04063103525990578
  1.7876303361723063
 -1.6264347964407893

In [31]:
@btime res = Optim.optimize(Optim.only_fg!(negative_log_likelihood!($S, $H, $X)), $coeff, 
            Optim.LBFGS(; m = 20, P = Preconditioner($precond), precondprep = (P, x) -> Preconditioner(precond!($S, $H, $X)($P.P.data, x))))

  72.319 ms (38554 allocations: 68.59 MiB)


 * Status: success

 * Candidate solution
    Final objective value:     1.479295e+00

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 8.58e-05 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.26e-05 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.74e-12 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.18e-12 ≰ 0.0e+00
    |g(x)|                 = 5.12e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    8
    f(x) calls:    17
    ∇f(x) calls:   17


In [32]:
@btime res = Optim.optimize(Optim.only_fg!(negative_log_likelihood!($S, $H, $X)), $coeff, 
            Optim.LBFGS(P = Preconditioner($precond)))#, precondprep = (P, x) -> Preconditioner(precond!($S, $H, $X)($P.P.data, x))))

  19.128 ms (26245 allocations: 46.64 MiB)


 * Status: success

 * Candidate solution
    Final objective value:     1.479295e+00

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 8.58e-05 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.26e-05 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.74e-12 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.18e-12 ≰ 0.0e+00
    |g(x)|                 = 5.12e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    8
    f(x) calls:    17
    ∇f(x) calls:   17


In [33]:
@btime res = Optim.optimize(Optim.only_fg!(negative_log_likelihood!($S, $H, $X)), $coeff, 
            Optim.LBFGS(; m = 20, P = Preconditioner($precond)))#, precondprep = (P, x) -> Preconditioner(precond!(S, H, X)(P.P.data, x))))

  19.342 ms (26265 allocations: 46.64 MiB)


 * Status: success

 * Candidate solution
    Final objective value:     1.479295e+00

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 8.58e-05 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.26e-05 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.74e-12 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.18e-12 ≰ 0.0e+00
    |g(x)|                 = 5.12e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    8
    f(x) calls:    17
    ∇f(x) calls:   17


In [34]:
@btime res = Optim.optimize(Optim.only_fg!(negative_log_likelihood!($S, $H, $X)), $coeff, 
            Optim.LBFGS(; m = 20, P = Preconditioner($precond)))

  19.304 ms (26265 allocations: 46.64 MiB)


 * Status: success

 * Candidate solution
    Final objective value:     1.479295e+00

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 8.58e-05 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.26e-05 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.74e-12 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.18e-12 ≰ 0.0e+00
    |g(x)|                 = 5.12e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    8
    f(x) calls:    17
    ∇f(x) calls:   17


In [39]:
diagprecond = zeros(Nψ)
coeff0 = randn(Nψ)
diagprecond!(diagprecond, coeff, S, H, X)
D = Optim.InverseDiagonal(diagprecond)

ldiv!(out::Array, P::Optim.InverseDiagonal, A::Array) = copyto!(out, A .* P.diag)
dot(A::Array, P::Optim.InverseDiagonal, B::Vector) = dot(A, B ./ P.diag)

dot (generic function with 40 methods)

In [41]:
diagprecond

14-element Array{Float64,1}:
 0.00012403916115351907
 0.000914061082337064
 4.223509631951479e-5
 0.0010497362861250537
 9.846338287295053e-5
 0.0023101801954228643
 0.006057967285600174
 0.000385561839624177
 0.0005285393135169043
 0.003055127424459309
 6.995783516676906e-5
 0.003976096151961689
 0.0004067998335964076
 0.000124219015787626

In [44]:
res = Optim.optimize(Optim.only_fg!(negative_log_likelihood!(S, H, X)), coeff, 
            Optim.LBFGS(; m = 20, P = Diagonal(diagprecond)))

 * Status: success

 * Candidate solution
    Final objective value:     1.479295e+00

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 6.88e-05 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.01e-05 ≰ 0.0e+00
    |f(x) - f(x')|         = 8.34e-13 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 5.63e-13 ≰ 0.0e+00
    |g(x)|                 = 9.72e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    11
    f(x) calls:    23
    ∇f(x) calls:   23


In [45]:
@btime res = Optim.optimize(Optim.only_fg!(negative_log_likelihood!($S, $H, $X)), $coeff, 
            Optim.LBFGS(; m = 20, P = Diagonal($diagprecond)))

  25.826 ms (35493 allocations: 63.09 MiB)


 * Status: success

 * Candidate solution
    Final objective value:     1.479295e+00

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 6.88e-05 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.01e-05 ≰ 0.0e+00
    |f(x) - f(x')|         = 8.34e-13 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 5.63e-13 ≰ 0.0e+00
    |g(x)|                 = 9.72e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    11
    f(x) calls:    23
    ∇f(x) calls:   23


In [37]:
res = Optim.optimize(Optim.only_fg!(negative_log_likelihood!(S, H, X)), zeros(Nψ), Optim.NGMRES())

 * Status: success

 * Candidate solution
    Final objective value:     1.479295e+00

 * Found with
    Algorithm:     Nonlinear GMRES preconditioned with Gradient Descent

 * Convergence measures
    |x - x'|               = 3.87e-05 ≰ 0.0e+00
    |x - x'|/|x'|          = 5.68e-06 ≰ 0.0e+00
    |f(x) - f(x')|         = 2.85e-13 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.93e-13 ≰ 0.0e+00
    |g(x)|                 = 5.95e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   1  (vs limit Inf)
    Iterations:    32
    f(x) calls:    123
    ∇f(x) calls:   123


In [128]:
@btime res = Optim.optimize(Optim.only_fg!(negative_log_likelihood!($S, $H, $X)), $coeff, Optim.AcceleratedGradientDescent())

  3.519 s (3527117 allocations: 6.96 GiB)


 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Final objective value:     1.572975e+00

 * Found with
    Algorithm:     Accelerated Gradient Descent

 * Convergence measures
    |x - x'|               = 4.30e-04 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.07e-04 ≰ 0.0e+00
    |f(x) - f(x')|         = 2.20e-09 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.40e-09 ≰ 0.0e+00
    |g(x)|                 = 1.35e-05 ≰ 1.0e-08

 * Work counters
    Seconds run:   4  (vs limit Inf)
    Iterations:    1000
    f(x) calls:    3295
    ∇f(x) calls:   3295


In [ ]:
MomentumGradientDescent()

In [30]:
X = randn(Nx, Ne) .* randn(Nx, Ne) 
S = Storage(H.I.f, X);

J = 0.0
dJ = zeros(Nψ)
d2J = zeros(Nψ, Nψ)
hess_negative_log_likelihood!(J, dJ, d2J, coeff, S, H, X)

1.3784285990365932

In [117]:
dJ

11-element Array{Float64,1}:
 -0.647941708993688
  0.01751723817324542
  0.12943970119038775
  0.0033091844731214684
 -0.00031264752126695754
  0.03545495128594644
 -0.0062483204244866645
 -0.028569877746525366
 -0.0008423358431159828
 -0.0033284486942029698
 -0.005733695198669693

In [31]:
@btime res = Optim.optimize(Optim.only_fgh!(hess_negative_log_likelihood!($S, $H, $X)), $coeff, Optim.NewtonTrustRegion())
# coeffopt = Optim.minimizer(res)

  193.793 ms (82746 allocations: 143.66 MiB)


 * Status: success

 * Candidate solution
    Final objective value:     1.341428e+00

 * Found with
    Algorithm:     Newton's Method (Trust Region)

 * Convergence measures
    |x - x'|               = 1.16e-05 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.01e-06 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.38e-13 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.03e-13 ≰ 0.0e+00
    |g(x)|                 = 2.62e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    28
    f(x) calls:    29
    ∇f(x) calls:   29
    ∇²f(x) calls:  23


In [40]:
@time res = Optim.optimize(Optim.only_fgh!(hess_negative_log_likelihood!(S, H, X)), coeff, Optim.NewtonTrustRegion())

  0.025023 seconds (17.23 k allocations: 44.434 MiB, 25.59% gc time)


 * Status: success

 * Candidate solution
    Final objective value:     1.450904e+00

 * Found with
    Algorithm:     Newton's Method (Trust Region)

 * Convergence measures
    |x - x'|               = 3.17e-04 ≰ 0.0e+00
    |x - x'|/|x'|          = 2.29e-04 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.02e-11 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 7.03e-12 ≰ 0.0e+00
    |g(x)|                 = 1.89e-13 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    4
    f(x) calls:    5
    ∇f(x) calls:   5
    ∇²f(x) calls:  4


In [121]:
res = Optim.optimize(Optim.only_fg!(negative_log_likelihood!(S, H, X)), coeff, Optim.LBFGS())
# coeffopt = Optim.minimizer(res)

 * Status: success

 * Candidate solution
    Final objective value:     1.298813e+00

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 4.47e-06 ≰ 0.0e+00
    |x - x'|/|x'|          = 4.06e-07 ≰ 0.0e+00
    |f(x) - f(x')|         = 5.15e-14 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 3.97e-14 ≰ 0.0e+00
    |g(x)|                 = 6.45e-10 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    19
    f(x) calls:    58
    ∇f(x) calls:   58


In [68]:
@time negative_log_likelihood!(J, dJ, coeff, S, H, X)

  0.000169 seconds (793 allocations: 47.328 KiB)


2.137129544913297